In [1]:
import requests
from datetime import datetime, timedelta
import pandas as pd
import time

# User-defined start and end dates
START_DATE = "2025-04-03"  # Format: YYYY-MM-DD
END_DATE = "2025-04-14"    # Format: YYYY-MM-DD

# API key and endpoints
API_KEY = "fcde36df-35d2-4feb-84ea-5b8faaa9baf6"
map_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/map"
ohlcv_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical"
headers = {"Accepts": "application/json", "X-CMC_PRO_API_KEY": API_KEY}

# List of cryptocurrencies
coins = [
    "ZEC", "JASMY", "FTT", "STPT", "ARK", "ARDR", "GPS", "MBL", "PERP", "NKN", "WING", "LTO", "FLM", "BSW", "ALPACA", "VOXEL", "PDA", "BTC"
]

# Step 1: Fetch coin IDs and names
def get_coin_info(symbols):
    params = {"symbol": ",".join(symbols)}
    try:
        response = requests.get(map_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        coin_map = {}
        for coin in data["data"]:
            symbol = coin["symbol"]
            if symbol in symbols:  # Only include requested symbols
                if symbol in coin_map:
                    if symbol == "BTC" and coin["name"] != "Bitcoin":
                        continue
                    if symbol == "BTC" and coin_map[symbol]["name"] != "Bitcoin":
                        coin_map[symbol] = {"id": coin["id"], "name": coin["name"]}
                else:
                    coin_map[symbol] = {"id": coin["id"], "name": coin["name"]}
        # Warn about missing symbols
        missing = [s for s in symbols if s not in coin_map]
        if missing:
            print(f"Warning: Symbols not found: {missing}")
        return coin_map
    except Exception as e:
        print(f"Error fetching coin info: {e}")
        return {}

# Step 2: Fetch historical data
def fetch_data(coin_map, start_date_str, end_date_str):
    end_time = datetime.strptime(end_date_str, "%Y-%m-%d").replace(hour=0, minute=0, second=0, microsecond=0)
    start_time_base = datetime.strptime(start_date_str, "%Y-%m-%d").replace(hour=0, minute=0, second=0, microsecond=0)
    start_time = start_time_base - timedelta(hours=1)  # Ensure API includes start date

    delta = (end_time - start_time_base).days + 1
    dates = [start_time_base + timedelta(days=x) for x in range(delta)]
    date_strings = [d.strftime("%Y-%m-%d") for d in dates]

    data_dict = {date: {symbol: None for symbol in coin_map} for date in date_strings}

    for symbol, info in coin_map.items():
        params = {
            "id": info["id"],
            "time_start": start_time.isoformat(),
            "time_end": end_time.isoformat(),
            "interval": "1d",
            "convert": "USD"
        }
        try:
            response = requests.get(ohlcv_url, headers=headers, params=params)
            response.raise_for_status()
            quotes = response.json()["data"]["quotes"]
            for quote in quotes:
                date = quote["time_open"].split("T")[0]
                if date in data_dict:
                    data_dict[date][symbol] = float(quote["quote"]["USD"]["close"])
            time.sleep(2)  # API rate limit delay
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    return data_dict, date_strings

# Main execution
coin_map = get_coin_info(coins)
if not coin_map:
    print("Failed to fetch coin info. Exiting.")
else:
    data_dict, date_strings = fetch_data(coin_map, START_DATE, END_DATE)

    # Create DataFrame with explicit column ordering
    columns = ["Date"] + [coin_map[symbol]["name"] for symbol in coins if symbol in coin_map]
    df = pd.DataFrame({"Date": date_strings})

    for symbol in coins:
        if symbol in coin_map:
            name = coin_map[symbol]["name"]
            df[name] = [
                data_dict[date][symbol] if data_dict[date][symbol] is not None else pd.NA
                for date in date_strings
            ]

    # Insert symbols row
    symbols_row = [""] + [symbol for symbol in coins if symbol in coin_map]
    symbols_df = pd.DataFrame([symbols_row], columns=columns)
    df = pd.concat([symbols_df, df], ignore_index=True)

    # Export to Excel with numeric formatting
    with pd.ExcelWriter("crypto_data_coinmarketcap.xlsx", engine="openpyxl") as writer:
        df.to_excel(writer, index=False, sheet_name="Sheet1")
        worksheet = writer.sheets["Sheet1"]
        num_data_rows = len(date_strings)
        for col in range(2, len(columns) + 1):
            for row in range(3, num_data_rows + 3):
                worksheet.cell(row=row, column=col).number_format = "0.00"

    print("Data exported to crypto_data_coinmarketcap.xlsx")

Data exported to crypto_data_coinmarketcap.xlsx
